In [2]:
# imports
import pandas as pd
import numpy as np

# SQL
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, inspect
from sqlalchemy.sql import text

In [3]:
# create filepath
filepath = "Resources/meteorite-landings.csv"

# read the CSV file
raw_df = pd.read_csv(filepath, encoding='latin1')

In [4]:
raw_df.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"


In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45716 entries, 0 to 45715
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         45716 non-null  object 
 1   id           45716 non-null  int64  
 2   nametype     45716 non-null  object 
 3   recclass     45716 non-null  object 
 4   mass         45585 non-null  float64
 5   fall         45716 non-null  object 
 6   year         45428 non-null  float64
 7   reclat       38401 non-null  float64
 8   reclong      38401 non-null  float64
 9   GeoLocation  38401 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 3.5+ MB


In [6]:
# remove null values 
raw_df.dropna(inplace=True)

In [7]:
raw_df.columns

Index(['name', 'id', 'nametype', 'recclass', 'mass', 'fall', 'year', 'reclat',
       'reclong', 'GeoLocation'],
      dtype='object')

In [8]:
# rename reclat and reclong for readability
raw_df.rename(columns={'reclat': 'lat', 'reclong': 'long'}, inplace=True)

In [9]:
# drop unnessary columns
cols = ['name', 'id', 'mass', 'recclass', 'year', 'lat', 'long', 'GeoLocation']
df = raw_df.loc[:, cols]
df.head()

,name,id,mass,recclass,year,lat,long,GeoLocation
0,Aachen,1,21.0,L5,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,720.0,H6,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,107000.0,EH4,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,1914.0,Acapulcoite,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,780.0,L6,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38116 entries, 0 to 45715
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         38116 non-null  object 
 1   id           38116 non-null  int64  
 2   mass         38116 non-null  float64
 3   recclass     38116 non-null  object 
 4   year         38116 non-null  float64
 5   lat          38116 non-null  float64
 6   long         38116 non-null  float64
 7   GeoLocation  38116 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 2.6+ MB


In [11]:
# create engine to sqlite
engine = create_engine("sqlite:///Resources/meteorite-landings.sqlite")
metadata = MetaData()

# Define the table schema with the primary key
table = Table(
    'meteorite-landings', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('mass', Float),
    Column('recclass', String),
    Column('year', Float),
    Column('lat', Float),
    Column('long', Float),
    Column('GeoLocation', String),
)

metadata.create_all(engine)

In [12]:
# write to the database
df.to_sql("meteorite-landings", con=engine, index=False, method="multi", if_exists="replace", chunksize=1000)

38116

In [13]:
# inspect to check it was created
inspector = inspect(engine)

# get names of tables within the database
tables = inspector.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    
    # get columns
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

meteorite-landings
name TEXT
id BIGINT
mass FLOAT
recclass TEXT
year FLOAT
lat FLOAT
long FLOAT
GeoLocation TEXT



In [14]:
df.head(10)

,name,id,mass,recclass,year,lat,long,GeoLocation
0,Aachen,1,21.0,L5,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,720.0,H6,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,107000.0,EH4,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,1914.0,Acapulcoite,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,780.0,L6,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"
5,Adhi Kot,379,4239.0,EH4,1919.0,32.10000,71.80000,"(32.100000, 71.800000)"
6,Adzhi-Bogdo (stone),390,910.0,LL3-6,1949.0,44.83333,95.16667,"(44.833330, 95.166670)"
7,Agen,392,30000.0,H5,1814.0,44.21667,0.61667,"(44.216670, 0.616670)"
8,Aguada,398,1620.0,L6,1930.0,-31.60000,-65.23333,"(-31.600000, -65.233330)"
9,Aguila Blanca,417,1440.0,L,1920.0,-30.86667,-64.55000,"(-30.866670, -64.550000)"
